In [7]:
def timetable(date1, date2 = 'not_date', to_json = False):
    import datetime
    import json
    list_b = list(map(int, date1.split('-')))
    date_b = datetime.date(list_b[0],list_b[1],list_b[2])
    if date2 == 'not_date':
        date_a = date_b
    else:
        list_a = list(map(int, date2.split('-')))
        date_a = datetime.date(list_a[0],list_a[1],list_a[2])
    import requests
    import time
    from bs4 import BeautifulSoup
    base_url = 'https://kinoteatr.ru/kinoafisha/?date='
    dic = {}
  
    for i in range((date_a - date_b).days + 1):
        r = requests.get(f'{base_url}{date_b}')
        soap = BeautifulSoup(r.text, 'lxml')
        date_str = '{}{}{}'.format(date_b.day, date_b.month, date_b.year)
        dic[date_str] = {}
        for block in soap.findAll('div', class_='col-md-2 col-sm-6 col-xs-12 movie_card'):
            href = block.find('a', class_='movie_card_clickable_zone gtm-ec-list-item-movie')['href']
            name = block.find('span', class_='movie_card_header title').text.strip()
            dic[date_str][name] = {}
            film_r = requests.get(href)
            film_soap = BeautifulSoup(film_r.text, 'lxml')
            for tt in film_soap.findAll('div', class_='cinema'):
                c_name = tt.find('div', class_='name').text.strip()
                lsttime = []
                for time in tt.findAll('div', class_='time'):
                    t = time.text.strip()[0:5]
                    lsttime.append(t)
                dic[date_str][name][c_name] = lsttime
        date_b = date_b + datetime.timedelta(days=1)
    if to_json:
        with open('{}.json'.format(to_json), 'w') as f:
            json.dump(dic, f)
        return True 
    else:
        return dic
timetable('2020-11-26','2020-11-30')


{'26112020': {'Непосредственно Каха': {'Кронверк Синема Облака': ['21:10',
    '23:30'],
   'Кронверк Синема Вэйпарк': ['21:10', '23:25'],
   'Синема Парк Филион на Багратионовской': ['21:10', '23:25'],
   'Кино Оkkо Щёлковский': ['21:15', '23:30'],
   'Формула Кино Европа': ['21:25', '23:45'],
   'Синема Парк Глобал Сити на Южной': ['21:30', '23:45'],
   'Синема Парк Пятая Авеню на Октябрьском Поле': ['21:35', '23:50'],
   'Синема Парк Бутово Молл': ['21:35', '23:50'],
   'Синема Парк на Калужской': ['21:40', '23:55'],
   'Синема Парк МЕГА Химки': ['21:50', '00:05'],
   'Формула Кино Лефортово': ['21:50', '23:30'],
   'Синема Парк Зеленопарк': ['21:50', '00:05'],
   'Формула Кино Ладога': ['21:50', '00:10'],
   'Синема Парк Ривьера на Автозаводской': ['21:55', '00:10'],
   'Синема ПАРК Теплый стан': ['21:55', '00:10'],
   'Формула Кино на Полежаевской': ['22:25'],
   'Формула Кино на Мичуринском': ['22:30', '00:45'],
   'Кронверк Синема Семеновский': ['22:40'],
   'Формула Кино на Мож